# Azure AI Agent service - Custom Bing integration

<img src="https://learn.microsoft.com/en-us/azure/ai-services/agents/media/agent-service-the-glue.png" width=800>

> https://learn.microsoft.com/en-us/azure/ai-services/agents/

In [1]:
import datetime
import os
import sys
import time

from azure.ai.agents import AgentsClient
from azure.identity import DefaultAzureCredential
from azure.ai.agents.models import BingCustomSearchTool
from azure.ai.agents.models import ListSortOrder
from dotenv import load_dotenv

In [2]:
sys.version

'3.10.14 (main, May  6 2024, 19:42:50) [GCC 11.2.0]'

In [3]:
print(f"Today is {datetime.datetime.today().strftime('%d-%b-%Y %H:%M:%S')}")

Today is 16-Jun-2025 14:35:49


## Settings

In [4]:
load_dotenv("azure.env")

True

In [5]:
BING_CUSTOM_CONN_ID = os.environ["BING_CUSTOM_CONN_ID"]

In [6]:
project_client = AgentsClient(
    endpoint=os.getenv("PROJECT_ENDPOINT"),
    credential=DefaultAzureCredential(),
)

## Initialize Bing Custom Search tool with connection id and instance name

In [7]:
instance_name = "travel"

# Initialize Bing Custom Search tool with connection id and instance name
bing_custom_tool = BingCustomSearchTool(connection_id=BING_CUSTOM_CONN_ID,
                                        instance_name=instance_name)

Exemple of instance definition:

<img src="custom.jpg">

## Agent creation

In [8]:
agent = project_client.create_agent(
    model='gpt-4o',
    name="travel-planner-agent",
    instructions="You are a travel planner dedicated to helping visitors discover top attractions, find the best hotels and restaurants, explore must-see sights, book flight tickets, and navigate public transportation with ease.",
    tools=bing_custom_tool.definitions,
    description="Travel planner"
)
print(f"Created agent, ID: {agent.id}")

Created agent, ID: asst_dolAvlJdOEpl1974W8RCpjRx


## Examples

In [9]:
prompt="List me the top 10 things to visit in Milan according to tripadvisor"

In [10]:
# Step 1: Create a thread
thread = project_client.threads.create()
print(f"🧵 [1] Created thread | ID: {thread.id}")

# Step 2: Send the user's message
message = project_client.messages.create(thread_id=thread.id,
                                         role="user",
                                         content=prompt)
print(f"💬 [2] Created message | ID: {message.id}")

# Step 3: Start a run
run = project_client.runs.create(thread_id=thread.id,
                                 agent_id=agent.id)
print(f"🏃 [3] Created run | ID: {run.id}")

# Step 4: Poll run status
print("⏳ [4] Polling run status...")
while run.status in {"queued", "in_progress", "requires_action"}:
    time.sleep(1)
    run = project_client.runs.get(thread_id=thread.id, run_id=run.id)
    print(f"🔄 Status: {run.status}")

    if run.status == "failed":
        print(f"❌ Run failed | Error: {run.last_error}")
        break

# Step 5: Display agent responses
print("\n🤖 [5] Agent responses:")
messages = project_client.messages.list(thread_id=thread.id,
                                        order=ListSortOrder.ASCENDING)

for msg in messages:
    print("\033[1;34m")  # Blue text for agent messages
    if msg.text_messages:
        for text_message in msg.text_messages:
            print(f"💡 → {text_message.text.value}")
    if msg.url_citation_annotations:
        print("\033[1;32m")  # Green text for citations
        for annotation in msg.url_citation_annotations:
            print(
                f"🔗 URL Citation: [{annotation.url_citation.title}]({annotation.url_citation.url})"
            )

print("\033[0m")  # Reset color

🧵 [1] Created thread | ID: thread_riwzM6lmj8QdrYZ0oUySRgM4
💬 [2] Created message | ID: msg_EpooGgIjoCeiIVljPx2nHaq1
🏃 [3] Created run | ID: run_hoXX3wxZdvMWq4t41wUMSy75
⏳ [4] Polling run status...
🔄 Status: in_progress
🔄 Status: in_progress
🔄 Status: in_progress
🔄 Status: in_progress
🔄 Status: completed

🤖 [5] Agent responses:

💡 → List me the top 10 things to visit in Milan according to tripadvisor

💡 → Here are the top 10 attractions to visit in Milan according to Tripadvisor:

1. **Duomo di Milano (Milan Cathedral)** - A stunning gothic cathedral and symbol of Milan, with an option to visit the rooftop.
2. **Galleria Vittorio Emanuele II** - A luxurious 19th-century arcade with shops, restaurants, and breathtaking architecture.
3. **The Last Supper (Il Cenacolo)** - Home to Leonardo da Vinci’s iconic fresco, housed at Santa Maria delle Grazie.
4. **Castello Sforzesco** - A historic castle featuring museums and expansive grounds.
5. **Navigli District** - Renowned for its historic ca

In [11]:
prompt="What are the best hotels in Milan according to social networks?"

In [12]:
# Step 1: Create a thread
thread = project_client.threads.create()
print(f"🧵 [1] Created thread | ID: {thread.id}")

# Step 2: Send the user's message
message = project_client.messages.create(thread_id=thread.id,
                                         role="user",
                                         content=prompt)
print(f"💬 [2] Created message | ID: {message.id}")

# Step 3: Start a run
run = project_client.runs.create(thread_id=thread.id,
                                 agent_id=agent.id)
print(f"🏃 [3] Created run | ID: {run.id}")

# Step 4: Poll run status
print("⏳ [4] Polling run status...")
while run.status in {"queued", "in_progress", "requires_action"}:
    time.sleep(1)
    run = project_client.runs.get(thread_id=thread.id, run_id=run.id)
    print(f"🔄 Status: {run.status}")

    if run.status == "failed":
        print(f"❌ Run failed | Error: {run.last_error}")
        break

# Step 5: Display agent responses
print("\n🤖 [5] Agent responses:")
messages = project_client.messages.list(thread_id=thread.id,
                                        order=ListSortOrder.ASCENDING)

for msg in messages:
    print("\033[1;34m")  # Blue text for agent messages
    if msg.text_messages:
        for text_message in msg.text_messages:
            print(f"💡 → {text_message.text.value}")
    if msg.url_citation_annotations:
        print("\033[1;32m")  # Green text for citations
        for annotation in msg.url_citation_annotations:
            print(
                f"🔗 URL Citation: [{annotation.url_citation.title}]({annotation.url_citation.url})"
            )

print("\033[0m")  # Reset color

🧵 [1] Created thread | ID: thread_3lyZnbkskOl6YQ05CnYLMHGp
💬 [2] Created message | ID: msg_6usEmiS0lmYHFAN9BK8Lpbsm
🏃 [3] Created run | ID: run_z16tfoFq43ScEcCH0JcrXtkn
⏳ [4] Polling run status...
🔄 Status: in_progress
🔄 Status: in_progress
🔄 Status: in_progress
🔄 Status: completed

🤖 [5] Agent responses:

💡 → What are the best hotels in Milan according to social networks?

💡 → The top hotels in Milan, frequently praised on social networks and hospitality platforms, include:

1. **Mandarin Oriental, Milan** - Renowned for its luxurious rooms and excellent location near city landmarks.
2. **Room Mate Giulia** - A boutique hotel appreciated for its vibrant design and proximity to the Duomo.
3. **Armani Hotel Milano** - Known for a chic aesthetic and exceptional service.
4. **Hotel Spadari al Duomo** - Offers a trendy stay close to cultural sites.
5. **Hotel Principe di Savoia** - A luxurious classic combining comfort and grandeur【3:0†source】【3:4†source】. 

These are popular not only for 

In [13]:
prompt = "Find some flights from Paris to Milan"

In [14]:
# Step 1: Create a thread
thread = project_client.threads.create()
print(f"🧵 [1] Created thread | ID: {thread.id}")

# Step 2: Send the user's message
message = project_client.messages.create(thread_id=thread.id,
                                         role="user",
                                         content=prompt)
print(f"💬 [2] Created message | ID: {message.id}")

# Step 3: Start a run
run = project_client.runs.create(thread_id=thread.id,
                                 agent_id=agent.id)
print(f"🏃 [3] Created run | ID: {run.id}")

# Step 4: Poll run status
print("⏳ [4] Polling run status...")
while run.status in {"queued", "in_progress", "requires_action"}:
    time.sleep(1)
    run = project_client.runs.get(thread_id=thread.id, run_id=run.id)
    print(f"🔄 Status: {run.status}")

    if run.status == "failed":
        print(f"❌ Run failed | Error: {run.last_error}")
        break

# Step 5: Display agent responses
print("\n🤖 [5] Agent responses:")
messages = project_client.messages.list(thread_id=thread.id,
                                        order=ListSortOrder.ASCENDING)

for msg in messages:
    print("\033[1;34m")  # Blue text for agent messages
    if msg.text_messages:
        for text_message in msg.text_messages:
            print(f"💡 → {text_message.text.value}")
    if msg.url_citation_annotations:
        print("\033[1;32m")  # Green text for citations
        for annotation in msg.url_citation_annotations:
            print(
                f"🔗 URL Citation: [{annotation.url_citation.title}]({annotation.url_citation.url})"
            )

print("\033[0m")  # Reset color

🧵 [1] Created thread | ID: thread_FryPjdExZ6Oy10ox6NXStKbm
💬 [2] Created message | ID: msg_HnoGtPjMLFSMFofsbQCIU8nx
🏃 [3] Created run | ID: run_R1DbYHadJcMcX2ERPQGcRycG
⏳ [4] Polling run status...
🔄 Status: in_progress
🔄 Status: in_progress
🔄 Status: in_progress
🔄 Status: completed

🤖 [5] Agent responses:

💡 → Find some flights from Paris to Milan

💡 → Flight options from Paris to Milan typically range from budget-friendly airlines to luxury ones, with prices starting as low as €25 for direct flights. Companies like Ryanair, Wizz Air, and easyJet offer competitive rates. Additionally, Air France and ITA Airways provide more premium services on this route. 

Flight durations are around 1.5 hours, and Paris has multiple departure points (Charles de Gaulle, Orly, and Beauvais airports) while Milan has Linate and Malpensa as arrival destinations. For specific dates and availability, it's advised to check airlines such as Ryanair, easyJet, or platforms like Kayak, BA, or Air France【3:1†sour

In [15]:
prompt = "What are the latest news from the city of Milan?"

In [16]:
# Step 1: Create a thread
thread = project_client.threads.create()
print(f"🧵 [1] Created thread | ID: {thread.id}")

# Step 2: Send the user's message
message = project_client.messages.create(thread_id=thread.id,
                                         role="user",
                                         content=prompt)
print(f"💬 [2] Created message | ID: {message.id}")

# Step 3: Start a run
run = project_client.runs.create(thread_id=thread.id,
                                 agent_id=agent.id)
print(f"🏃 [3] Created run | ID: {run.id}")

# Step 4: Poll run status
print("⏳ [4] Polling run status...")
while run.status in {"queued", "in_progress", "requires_action"}:
    time.sleep(1)
    run = project_client.runs.get(thread_id=thread.id, run_id=run.id)
    print(f"🔄 Status: {run.status}")

    if run.status == "failed":
        print(f"❌ Run failed | Error: {run.last_error}")
        break

# Step 5: Display agent responses
print("\n🤖 [5] Agent responses:")
messages = project_client.messages.list(thread_id=thread.id,
                                        order=ListSortOrder.ASCENDING)

for msg in messages:
    print("\033[1;34m")  # Blue text for agent messages
    if msg.text_messages:
        for text_message in msg.text_messages:
            print(f"💡 → {text_message.text.value}")
    if msg.url_citation_annotations:
        print("\033[1;32m")  # Green text for citations
        for annotation in msg.url_citation_annotations:
            print(
                f"🔗 URL Citation: [{annotation.url_citation.title}]({annotation.url_citation.url})"
            )

print("\033[0m")  # Reset color

🧵 [1] Created thread | ID: thread_cXhDs4G5Ws0z4e8ChVVOHRV6
💬 [2] Created message | ID: msg_o23a6jaVd8UYVSTOgZoCMfwT
🏃 [3] Created run | ID: run_dUTgJnQGHPmMMVRCYi58UEwO
⏳ [4] Polling run status...
🔄 Status: in_progress
🔄 Status: in_progress
🔄 Status: in_progress
🔄 Status: in_progress
🔄 Status: completed

🤖 [5] Agent responses:

💡 → What are the latest news from the city of Milan?

💡 → Here are some recent updates from Milan:

1. **Public Transport and Train Service Issues**:
   - The train line between Milan and Varenna will face disruptions due to construction work, starting 15 June 2025 and lasting until 14 September 2025. Replacement bus services will be available【3:0†source】【3:2†source】.
   - A General Strike is scheduled for 20 June 2025, which may disrupt metro and other public transportation systems in Milan【3:1†source】.

2. **Cultural Updates**:
   - Tickets for "The Last Supper" experience are highly sought after; advance bookings are recommended as reservations open several m

## Post processing

In [18]:
print(
    f"Agent Details:\n"
    f"🆔 ID          : {agent.id}\n"
    f"📛 Name        : {agent.name}\n"
    f"🧠 Model       : {agent.model}\n"
    f"📜 Instructions: {agent.instructions}"
)

Agent Details:
🆔 ID          : asst_dolAvlJdOEpl1974W8RCpjRx
📛 Name        : travel-planner-agent
🧠 Model       : gpt-4o
📜 Instructions: You are a travel planner dedicated to helping visitors discover top attractions, find the best hotels and restaurants, explore must-see sights, book flight tickets, and navigate public transportation with ease.


In [ ]:
#print("Deleting the agent we have created...")
#project_client.delete_agent(agent.id)
#print(f"Deleted agent, agent ID: {agent.id}")
#print("Done")